## 生成FDA Drug Label的关系数据

In [1]:
import pandas as pd
import os
import json
import re
os.getcwd()

'D:\\drug KG\\pgkb'

In [2]:
df_drug_label = pd.read_csv('drug_label/drugLabels.tsv', sep='\t').fillna("")
df_drug_label = df_drug_label[df_drug_label["Source"] == "FDA"]
print(df_drug_label.shape)
df_drug_label[:10]

(373, 14)


,PharmGKB ID,Name,Source,Biomarker Flag,Testing Level,Has Prescribing Info,Has Dosing Info,Has Alternate Drug,Cancer Genome,Prescribing,Chemicals,Genes,Variants/Haplotypes,Latest History Date (YYYY-MM-DD)
28,PA166184637,Annotation of FDA Label for oxymetazoline and ...,FDA,On,Actionable PGx,Prescribing Info,,,,Prescribing,oxymetazoline and tetracaine,BCHE; CYB5R3; G6PD,,2019-10-24
31,PA166153492,Annotation of FDA Label for brivaracetam and C...,FDA,On,Actionable PGx,Prescribing Info,,,,Prescribing,brivaracetam,CYP2C19,,2017-11-09
33,PA166178724,Annotation of FDA Label for neratinib and ESR1...,FDA,On,Informative PGx,,,,,,neratinib,ESR1; ESR2; PGR,,
34,PA166163421,Annotation of FDA Label for daclatasvir and IFNL3,FDA,On,Informative PGx,,,,,,daclatasvir,IFNL3,,
35,PA166160039,Annotation of FDA Label for evolocumab and LDLR,FDA,Off (Formerly On),Informative PGx,,,,,,evolocumab,LDLR,,2017-11-08
36,PA166160040,Annotation of FDA Label for Parathyroid Hormon...,FDA,On,Informative PGx,,,,,,Parathyroid Hormones And Analogues,CASR,,
38,PA166105195,Annotation of FDA Label for metoclopramide and...,FDA,On,Actionable PGx,,,,,,metoclopramide,CYB5R1; CYB5R2; CYB5R3; CYB5R4; G6PD,,2019-10-17
39,PA166159962,Annotation of FDA Label for alirocumab and LDLR,FDA,Off (Formerly On),Informative PGx,,,,,,alirocumab,LDLR,,2017-11-08
40,PA166105211,Annotation of FDA Label for ondansetron and CY...,FDA,On,Informative PGx,,,,,,ondansetron,CYP2D6,,
45,PA166182786,Annotation of FDA Label for tolazamide and G6PD,FDA,On,Actionable PGx,Prescribing Info,,,,Prescribing,tolazamide,G6PD,,


In [4]:
drug_gene_label_list = []
drug_variant_label_list = []

for index, row in df_drug_label.iterrows():
    genes = row["Genes"]
    gene_list = [x.strip() for x in genes.split(";")]
    variant = row["Variants/Haplotypes"]
    variant_list = filter(lambda x: x!= "", [x.strip() for x in variant.split(";")])
    chemicals = row["Chemicals"]
    chemical_list = filter(lambda x: x!= "", [x.strip() for x in re.split(r"and|/", chemicals)])
    source = row["Source"]
    level = row["Testing Level"]
    
    for c in chemical_list:
        for g in gene_list:
            drug_gene_label_list.append((c, level, source, g))
            
        for v in variant_list:
            drug_variant_label_list.append((c, level, source, v))


In [5]:
print(len(drug_variant_label_list))
drug_variant_label_list[:2]

118


[('simeprevir', 'Informative PGx', 'FDA', 'rs12979860'),
 ('ponatinib', 'Testing required', 'FDA', 'rs121913459')]

In [6]:
print(len(drug_gene_label_list))
drug_gene_label_list[:2]

636


[('oxymetazoline', 'Actionable PGx', 'FDA', 'BCHE'),
 ('oxymetazoline', 'Actionable PGx', 'FDA', 'CYB5R3')]

In [9]:
pd.DataFrame(drug_gene_label_list, 
             columns=["drug", "label", "organization", "gene"]).assign(
    data_source=["drug_label"] * len(drug_gene_label_list)).to_csv("processed/fda_drug_gene_label.csv", index=False)

pd.DataFrame(drug_variant_label_list, 
             columns=["drug", "label", "organization", "variant"]).assign(
    data_source=["drug_label"] * len(drug_variant_label_list)).to_csv("processed/fda_drug_variant_label.csv", index=False)